In [1]:
using BenchmarkTools

In [8]:
module My

struct IndexSetCache{T}
    cache::Dict{BigInt,T}
    coeffs::Vector{BigInt}
    function IndexSetCache{T}(dims) where T
        coeffs = ones(BigInt, length(dims))
        for n in 2:length(dims)
            coeffs[n] = dims[n-1] * coeffs[n-1]
        end
        new(Dict{BigInt,T}(), coeffs)
    end
end

function (cf::IndexSetCache{T})(
    x::Vector{Int}
) where {T}
    length(x) == length(cf.coeffs) || error("Invalid length of x")
    return get!(cf.d, _key(cf, x)) do
        cf.f(x)
    end
end

end

Main.My

In [11]:
import .My

localdims = fill(2, 20)
cache = My.IndexSetCache{Int}(localdims)

Main.My.IndexSetCache{Int64}(Dict{BigInt, Int64}(), BigInt[1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144, 524288])

In [ ]:
tolerance = 1e-12
localdims = localdims1 .* localdims3
tci, ranks, errors =
    TCI.crossinterpolate2(Float64, ab, localdims, [ones(Int, N)]; tolerance = tolerance, loginterval=1)

In [ ]:
sitesa = [Index(d, "a=$i") for (i, d) in enumerate(localdims1)]
sitesb = [Index(d, "b=$i") for (i, d) in enumerate(localdims3)]
M = Quantics.TCItoMPS(tci)
M = Quantics.unfuse_siteinds_nosplit(M, siteinds(M), [collect(i) for i in zip(sitesa, sitesb)])

In [ ]:
ab_ref = contract(ab.a_MPO, ab.b_MPO; alg="naive")
for n in eachindex(ab_ref)
    ab_ref[n] = replaceind(ab_ref[n], ab.sites1[n], sitesa[n])
    ab_ref[n] = replaceind(ab_ref[n], ab.sites3[n], sitesb[n])
end
ab_ref[1] *= onehot(ab.links_a[1] => 1)
ab_ref[1] *= onehot(ab.links_b[1] => 1)
ab_ref[end] *= onehot(ab.links_a[end] => 1)
ab_ref[end] *= onehot(ab.links_b[end] => 1)
ab_ref

In [ ]:
ab_ref ≈ M